<a href="https://colab.research.google.com/github/Anshuman-02905/Custom_Satellite_Scripts/blob/main/getImages_and_Masks__ver4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%%capture
!pip install geemap
!pip install geopandas
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y

In [22]:
import geemap
import os
import glob
import gdal ## fingers crossed!
import os
import ee
import pandas as pd
import numpy as np
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
import os

In [23]:
os.mkdir("Sentinel_png")
os.mkdir("Mask_png")
os.mkdir("Sentinel_tif")
os.mkdir("Urban_tif")


FileExistsError: ignored

In [35]:
Sentinel_tif="/content/Sentinel_tif/"
urban_tif="/content/Urban_tif/"
Sentinel_png="/content/Sentinel_png/"
Mask_png="/content/Mask_png/"

In [26]:
Map=geemap.Map() 

In [ ]:
Map=geemap.Map()
state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bangalore Urban'))
sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(state)
image = sentinel.mean().divide(10000).clip(state).select(['B4', 'B3', 'B2']).unmask()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
  }
#Map.addLayer(sentinel.mean().divide(10000).clip(state), visualization, 'RGB')

mask = ee.Image("DLR/WSF/WSF2015/v1").clip(state)

visualization = {
  'min': 0,
  'max': 255,
}

#Map.addLayer(urban.clip(state), visualization, "Human1")
mask=mask.expression('b("settlement") == 255 ? 255:0',{'settlement':mask.select('settlement')}).clip(state)

Map.addLayer(mask, visualization, "Human2")
Map.centerObject(state)
Sentinel="/content/Sentinel_tif/"+"sentinel.tif"
urban="/content/Urban_tif/"+"final.tif"

geemap.ee_export_image(mask, filename=urban, scale=50, region=state.geometry(), file_per_band=False)
geemap.ee_export_image(image, filename=Sentinel, scale=50, region=state.geometry(), file_per_band=False)
text1="/content/Sentinel_png/"+"sentinel"+".png"
text2="/content/Mask_png/"+"final"+".png"
options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
options_string = " ".join(options_list) 
gdal.Translate(text1,Sentinel, options=options_string)
options_list = ['-ot Byte','-of PNG'] 
options_string = " ".join(options_list)
gdal.Translate(text2, urban, options=options_string) 




Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/final.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Sentinel_tif/sentinel.tif


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f87994ac720> >

In [ ]:
#state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bangalore Urban'))
#shps=state.getInfo()['features'][0]['geometry']['coordinates']

In [ ]:
#state.getInfo()['features'][0]['geometry']['coordinates']

In [ ]:
'''final_list = [item for sublist in shps for item in sublist]
min_lat=1000;
min_lon=1000;
max_lat=-1000;
max_lon=-1000;
for i in final_list:
  min_lat=min(min_lat,i[0])
  min_lon=min(min_lon,i[1])
  max_lat=max(max_lat,i[0])
  max_lon=max(max_lon,i[1])
min_lon-=0.00098
max_lon+=0.00998
left_upper=tuple([min_lat,min_lon])
left_lower=tuple([max_lat,min_lon])
right_lower=tuple([max_lat,max_lon])
right_upper=tuple([min_lat,max_lon])'''

In [ ]:
#def get_distance(LA1,LN1,LA2,LN2):
#  return math.sqrt(math.pow((LA1-LA2),2)+math.pow((LN1-LN2),2))

In [9]:
#left_dist=get_distance(left_upper[0],left_upper[1],left_lower[0],left_lower[1])
#print(left_dist)
#right_dist=get_distance(right_upper[0],right_upper[1],right_lower[0],right_lower[1])
#print(right_dist)
#upper_dist=get_distance(left_upper[0],left_upper[1],right_upper[0],right_upper[1])
#print(upper_dist)
#lower_dist=get_distance(left_lower[0],left_lower[1],right_lower[0],right_lower[1])
#print(lower_dist)
left_dist=0.5033789690619983
right_dist=0.5033789690619983
upper_dist=0.5859878671830234
lower_dist=0.5859878671830234

In [29]:
final_list=list()
cord=[12.977200441508167, 77.58441571886084]
Lat=cord[0]
Lon=cord[1]
c=0
for i in range(0,1):
  Lon=cord[1]
  for j in range(0,2):
    LL=[Lon,Lat]
    UL=[Lon,Lat+left_dist]
    LR=[Lon+lower_dist,Lat]
    UR=[Lon+lower_dist,Lat+left_dist]
    final_list=[LL,LR,UR,UL,LL]
    result=get_boundaries(final_list,c)
    if result!=-1:
      c+=1
    print(final_list)
    Lon+=lower_dist
  Lat+=left_dist
 #Map.setCenter(cord[1],cord[0],10)
#Map
#print(final_list)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


local variable 'Sentinel' referenced before assignment
Exception
[[77.58441571886084, 12.977200441508167], [78.17040358604386, 12.977200441508167], [78.17040358604386, 13.480579410570165], [77.58441571886084, 13.480579410570165], [77.58441571886084, 12.977200441508167]]


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


local variable 'Sentinel' referenced before assignment
Exception
[[78.17040358604386, 12.977200441508167], [78.75639145322688, 12.977200441508167], [78.75639145322688, 13.480579410570165], [78.17040358604386, 13.480579410570165], [78.17040358604386, 12.977200441508167]]


In [ ]:
final_list=list()
#cord=[0, 0]
cord=[12.977200441508167, 77.58441571886084]

Lat=cord[0]
Lon=cord[1]
c=0
for i in range(0,1):
  Lon=cord[1]
  for j in range(0,4):
    LL=[Lon,Lat]
    UL=[Lon,Lat+left_dist]
    LR=[Lon+lower_dist,Lat]
    UR=[Lon+lower_dist,Lat+left_dist]
    final_list=[LL,LR,UR,UL,LL]
    result=get_boundaries(final_list,c)
    if result!=-1:
      c+=1
    print(final_list)
    print("Image_number",c)
    Lon+=lower_dist
  Lat+=left_dist
 #Map.setCenter(cord[1],cord[0],10)
#Map
#print(final_list)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Sentinel_tif/0.tif
[[77.58441571886084, 12.977200441508167], [78.17040358604386, 12.977200441508167], [78.17040358604386, 13.480579410570165], [77.58441571886084, 13.480579410570165], [77.58441571886084, 12.977200441508167]]
Image_number 1


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/1.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Sentinel_tif/1.tif


In [ ]:
'''final_list=[[ 77.58441571886084,12.977200441508167],
 [78.17040358604386,12.977200441508167],
 [78.17040358604386,13.480579410570165],
 [77.58441571886084,13.480579410570165],
 [77.58441571886084,12.977200441508167]]'''

In [ ]:
#print(final_list)

[[77.58441571886084, 12.977200441508167], [78.17040358604386, 12.977200441508167], [78.17040358604386, 13.480579410570165], [77.58441571886084, 13.480579410570165], [77.58441571886084, 12.977200441508167]]


In [36]:
def get_boundaries(final_list,c):
  coordinates = final_list
  def create_polygon(coords, polygon_name):
    ''' Create a polygon from coordinates'''
    polygon = Polygon(coordinates)
    gdf = gpd.GeoDataFrame(crs = {'init' :'epsg:4326'})
    gdf.loc[0,'name'] = polygon_name
    gdf.loc[0, 'geometry'] = polygon
    return gdf
  shapefile = create_polygon(coordinates, "Amesterdam")
  shapefile.to_file("newboundary.shp")
  mine_shp="/content/newboundary.shp"
  mine = geemap.shp_to_ee(mine_shp) 
  return func(c)
  #return -1

  

In [ ]:
print(get_boundaries(final_list,1))

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


-1


In [ ]:
'''mine_shp="/content/newboundary.shp"
mine = geemap.shp_to_ee(mine_shp) 
Map=geemap.Map()
state = mine
sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(state)
image = sentinel.mean().divide(10000).clip(state).select(['B4', 'B3', 'B2']).unmask()

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}
Map.addLayer(image, visualization, 'RGB')
Map.centerObject(state)
Map'''

In [34]:
def func(c):
  try:
    mine_shp="/content/newboundary.shp"
    mine = geemap.shp_to_ee(mine_shp) 
    Map=geemap.Map()
    state = mine
    #Map=geemap.Map()
    #state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bangalore Urban'))
    sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(state)
    image = sentinel.mean().divide(10000).clip(state).select(['B4', 'B3', 'B2']).unmask()

    visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
    }
    Map.addLayer(image, visualization, 'RGB')

    mask = ee.Image("DLR/WSF/WSF2015/v1").clip(state)

    visualization = {
    'min': 0,
    'max': 255,
    }

    #Map.addLayer(urban.clip(state), visualization, "Human1")
    mask=mask.expression('b("settlement") == 255 ? 255:0',{'settlement':mask.select('settlement')}).clip(state)

    Map.addLayer(mask, visualization, "Human2")
    Map.centerObject(image)
    Sentinel=Sentinel_tif+str(c)+".tif"
    urban=urban_tif+str(c)+".tif"

    geemap.ee_export_image(mask, filename=urban, scale=50, region=state.geometry(), file_per_band=False)
    geemap.ee_export_image(image, filename=Sentinel, scale=50, region=state.geometry(), file_per_band=False)
    Sentinel_mask=Sentinel_png+str(c)+".png"
    Mask_mask=Mask_png+str(c)+".png"
    options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
    options_string = " ".join(options_list) 
    gdal.Translate(Sentinel_mask,Sentinel, options=options_string)
    options_list = ['-ot Byte','-of PNG'] 
    options_string = " ".join(options_list)
    gdal.Translate(Mask_mask, urban, options=options_string)
    xml1=Sentinel_mask+".aux.xml"
    xml2=Mask_mask+".aux.xml"
    os.remove(xml1)
    os.remove(xml2)
    return 1
  except Exception as e:
    print(e)
    print("Exception")
    return -1


In [ ]:
Map

In [ ]:
final_list

[[0, 28.64068271262587],
 [0.5859878671830234, 28.64068271262587],
 [0.5859878671830234, 29.14406168168787],
 [0, 29.14406168168787],
 [0, 28.64068271262587]]